In [1]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/naver_comment_2.csv', encoding='utf-8-sig')
df.head()

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,웰캄투실버라이프,9.94,19878,15,아 할머니 귀여우시다 힐링툰 감이 오네요 잘볼게용!\n시리얼에 뜨거운 우유...나만...,52588,599,2220,정주행 시작\n솔찍히 자는데 깨우면 짜증남..할머니라도..맘은 이해하지만 더 자고 ...,9.98
1,원수를 사랑하라,9.95,31528,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129562,3792,987,근데 나 자신을 믿는것.. 지금 내가 제일 필요한 말임..ㅠㅠㅠㅠㅠ 어제 면접 말려...,9.92
2,약초마을 연쇄살초사건,9.73,10367,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,63294,255,1462,이거 왤케 재밌냨ㅋ 팀장님 죽었는데 뒷담 까는거 보소\n약초마을 연쇄살인도 아닌 연...,9.87
3,AI가 세상을 지배한다면,9.88,12207,15,그대가 POGO 싶었소\n라움 코드 자기가 넣었다고... 이리 당당하게 선전POGO...,59238,967,2036,완결보고 정주행만 3번째 볼때마다 감정이 다름 영화로 나오기엔 잘라내거나 스토리에 ...,9.94
4,닥터 프로스트 시즌 3~4,9.95,30579,15,아무생각없이 내렸다가 이거 보고 놀란사람 손!!!\n헐?\n제가 이 웹툰을 보면서 ...,119758,2817,9707,퍄ㅑㅑ\n닥프 1기 2기 무료로 풀려있으니까 꼭보세요\n아 이런 띵작은 유료되기전에...,9.97


In [3]:
def train_to_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    
    lgbm = LGBMClassifier()
    xgb = XGBClassifier()
    rf = RandomForestClassifier()
    lr = LogisticRegression()

    regs = [lgbm, xgb, rf, lr]

    result = []
    for reg in regs:
        reg.fit(X_train, y_train)
        pred = reg.predict(X_val)
        acc = accuracy_score(y_val, pred)
        print(reg.__class__.__name__)
        print(f'Accuracy: {acc:.4f}')
        
    return regs

In [4]:
# 9.9 이상, 미만 비율이 6:4
def category(star):
    if star >= 9.9:
        return 1
    else:
        return 0

# 1. 텍스트 제외 모든 feature

In [5]:
X = df.drop(columns=['title', 'best', 'comment', 'star'])
y = df['star'].apply(category)

regs1 = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.8304
[17:27:24] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.8125
RandomForestClassifier
Accuracy: 0.7946
LogisticRegression
Accuracy: 0.7232


# 2. 베댓 추천 수, 비추천 수 제외

In [6]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'best_recomm', 'best_unrecomm'])

regs2 = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.7857
[17:27:24] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.7500
RandomForestClassifier
Accuracy: 0.8036
LogisticRegression
Accuracy: 0.6786


## 추가적으로 베댓 수도 제외

In [7]:
X = X.drop(columns=['best_num'])

regs2_a = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.7857
[17:27:24] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.7500
RandomForestClassifier
Accuracy: 0.8125
LogisticRegression
Accuracy: 0.6786


## 베댓 수만 제외

In [8]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'best_num'])

regs2_b = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.8304
[17:27:25] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.8125
RandomForestClassifier
Accuracy: 0.8036
LogisticRegression
Accuracy: 0.7232


# 3. 1화 평점 제외

In [9]:
X = df.drop(columns=['title', 'best', 'comment', 'star', 'score'])

regs3 = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.7411
[17:27:25] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.6875
RandomForestClassifier
Accuracy: 0.7143
LogisticRegression
Accuracy: 0.7232


# 4. 1화 평점, 평가 인원으로만

In [10]:
X = df[['score', 'score_num']]

regs4 = train_to_evaluate(X, y)

LGBMClassifier
Accuracy: 0.8036
[17:27:25] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier
Accuracy: 0.7411
RandomForestClassifier
Accuracy: 0.7946
LogisticRegression
Accuracy: 0.6071


당연한 결과겠지만, 1화 평점이 평균 평점을 예측하는 데 있어 매우 중요한 요소이며 그렇다고 1화 평점만으로는 평균 평점을 예측하는 데 무리가 있다.  
그리고 베댓 수, 베댓 추천 수, 베댓 비추천 수는 오히려 오차를 늘리는 요소이므로 향후 최종 모델에서는 제외하는 것으로...  
또한 TF-IDF 결과보다 더 우수한 결과가 나왔다.